# Transformación de los datos 

In [1]:
# !pip install pyarrow

In [1]:
# Importar librerías
import pandas as pd
import os
import warnings
import re

In [69]:
# Crear carpeta para almacenar datos Tidy
if not os.path.exists('Tidy'):
    os.mkdir("Tidy")

## SIAP

### Unión de los datos

In [70]:
# Arreglos con valores de años, meses y cultivos
anio_arr = pd.read_csv('./Catalogos_API_SIAP/Anio_Agricola.csv').ID.to_list()
cultivo_arr = pd.read_csv('./Catalogos_API_SIAP/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./Catalogos_API_SIAP/Mes_Agricola.csv').ID.to_list()
cultivo_arr.remove(0) # 0: Todos

In [71]:
# Dataframe vacío con las columnas de nuestros datos
columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
df_raw = pd.DataFrame(columns=columnas)

total = 0
# Iteración sobre CSV's para llenar DataFrame
for anio in anio_arr:
    for i in range(1,13):
        df_aux = pd.read_csv("./raw/SIAP/"+str(anio)+"/Avance_Agricola_"+str(anio)+ "_" + str(i) + '.csv')
        df_raw = pd.concat([df_raw, df_aux], axis=0)
        total += df_aux.shape[0]

# La columna Superficie Siniestrada se llena con nan's en sus valores faltantes
df_raw['Superficie Siniestrada'] = pd.to_numeric(df_raw['Superficie Siniestrada'], errors='coerce')

C:\Users\jcbar\AppData\Local\Temp\ipykernel_12576\3475416861.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw = pd.concat([df_raw, df_aux], axis=0)


In [72]:
# Si aún no existe, guardamos el DataFrame en un parquet
if not os.path.exists('raw/SIAP/Avance_Agricola_Historico.parquet'):
    name_csv = "Avance_Agricola_Historico"
    ruta_completa = os.path.join("raw/SIAP/",name_csv)
    df_raw.to_parquet(ruta_completa + '.parquet', index=False)

In [73]:
# Instancias de los datos
print("Total de datos: ",total)

Total de datos:  1154774


### Decodificación de variables

In [74]:
# Cargar parquet
df = pd.read_parquet('./raw/SIAP/Avance_Agricola_Historico.parquet')

In [75]:
catalogo_cultivos = pd.read_csv('./Catalogos_API_SIAP/Cultivo.csv')
catalogo_mes = pd.read_csv('./Catalogos_API_SIAP/Mes_Agricola.csv')

catalogo_cultivos = catalogo_cultivos.set_index('ID')['Name'].to_dict()
catalogo_mes = catalogo_mes.set_index('ID')['Name'].to_dict()

df['Cultivo'] = df['Cultivo'].map(catalogo_cultivos)
df['Mes'] = df['Mes'].map(catalogo_mes)


In [76]:
if not os.path.exists('./Tidy/SIAP.parquet'):
    df.to_parquet('./Tidy/SIAP.parquet', index=False)

In [77]:
df = pd.read_parquet('./Tidy/SIAP.parquet')

In [78]:
df.columns.to_list()

['Entidad',
 'Municipio',
 'Superficie Sembrada',
 'Superficie Cosechada',
 'Superficie Siniestrada',
 'Produccion',
 'Rendimiento',
 'Anio',
 'Mes',
 'Cultivo']

In [79]:
df.shape

(1154774, 10)

In [80]:
df.sample(n=5)

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
792011,Sinaloa,Mocorito,"10,631.00",0.00,0.0,0.00,0,2019,Marzo,Maíz grano
916004,Nayarit,Compostela,90.00,90.00,0.0,"1,548.00",17.20,2019,Octubre,Sorgo forrajero en verde
916321,Guerrero,Huitzuco de Los Figueroa,537.83,0.00,0.0,0.00,0,2019,Octubre,Sorgo grano
816723,Oaxaca,Villa Díaz Ordaz,6.95,0.00,0.0,0.00,0,2019,Mayo,Durazno
513082,Guerrero,Arcelia,2.98,0.00,0.0,0.00,0,2021,Octubre,Guayaba


## SNIIM

### Unión de los datos

In [43]:
# Crea una lista para almacenar los DataFrames de los archivos CSV
dataframes = []

# Itera a través de los archivos en la ruta
for archivo in os.listdir("./raw/SNIIM/"):
    if archivo.endswith(".csv"):
        # Lee cada archivo CSV y lo convierte en un DataFrame
        archivo_csv = os.path.join('./raw/SNIIM/' + str(archivo))
        if os.path.getsize(archivo_csv) > 0:
            try:
                df_sniim = pd.read_csv(archivo_csv)
                dataframes.append(df_sniim)
            except pd.errors.EmptyDataError:
                print(f"El archivo {archivo} está vacío y se omitirá.")

El archivo sniim_product_aguacate_fuerte_primera.csv está vacío y se omitirá.
El archivo sniim_product_aguacate_hass_flor_vieja_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_16_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_45_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_48_primera.csv está vacío y se omitirá.
El archivo sniim_product_papa_marciana_primera.csv está vacío y se omitirá.
El archivo sniim_product_plátano_enanogigante_primera.csv está vacío y se omitirá.
El archivo sniim_product_santa_lucía_girasol.csv está vacío y se omitirá.
El archivo sniim_product_santa_lucía_vegetal_mixto.csv está vacío y se omitirá.
El archivo sniim_product_santísima_vegetal_mixto.csv está vacío y se omitirá.
El archivo sniim_product_sarita_cártamo.csv está vacío y se omitirá.
El archivo sniim_product_sarita_maíz.csv está vacío y se omitirá.
El archivo sniim_product_sarita_vegetal_mixto.csv está va

In [44]:
# Combina todos los DataFrames en uno solo
df_completo = pd.concat(dataframes, ignore_index=True)

In [45]:
# Renombrar las columnas 
nuevos_nombres = {
    "fecha": "Fecha",
    "presentacion": "Presentacion",
    "origen": "Origen",
    "destino": "Destino",
    "precio_min": "Precio_min",
    "precio_max": "Precio_max",
    "obs": "Observacion",
    "name":"Cultivo"
}

# Aplicar el renombramiento al df_completo
df_completo = df_completo.rename(columns=nuevos_nombres)



In [53]:
# Dimensión de los datos
print(f"Dimensión de los datos: {df_completo.shape}")

Dimensión de los datos: (900763, 9)


In [57]:
len(df_completo['Destino'].unique())

46

In [46]:
# Muestra de 5 filas aleatorias del dataframe
df_completo.sample(n=5)

,Fecha,Presentacion,Origen,Destino,Precio_min,Precio_max,precio_frec,Observacion,Cultivo
518265,26/04/2023,Caja de 28 kg.,Chiapas,Aguascalientes: Centro Comercial Agropecuario ...,23.21,23.93,23.57,NaN,mango ataulfo_primera
90893,22/09/2023,Kilogramo,Guanajuato,Guanajuato: Central de Abasto de León,28.00,30.0,28.00,NaN,camote_primera
214796,19/10/2021,Arpilla de 24 kg.,Baja California,"Baja California : Central de Abasto INDIA, Tij...",10.42,10.42,10.42,NaN,chile jalapeño_primera
142191,15/05/2018,Kilogramo,Jalisco,Baja California Sur: Unión de Comerciantes de ...,145.00,150.0,150.00,NaN,chile ancho_primera
174173,08/06/2022,Kilogramo,Sinaloa,"Coahuila: Central de Abasto de La Laguna, Torreón",25.00,28.0,28.00,NaN,chile chilaca_primera


In [47]:
# Tipo de dato de cada columna
df_completo.dtypes

Fecha            object
Presentacion     object
Origen           object
Destino          object
Precio_min      float64
Precio_max       object
precio_frec     float64
Observacion      object
Cultivo          object
dtype: object

In [48]:
# "Precio_max" de objecto a float
df_completo['Precio_max'] = df_completo['Precio_max'].str.replace(",", "")
df_completo['Precio_max'] = df_completo['Precio_max'].astype("float")
# Nuevo tipo de dato de cada columna
df_completo.dtypes

Fecha            object
Presentacion     object
Origen           object
Destino          object
Precio_min      float64
Precio_max      float64
precio_frec     float64
Observacion      object
Cultivo          object
dtype: object

In [49]:
# Almacenamos el archivo como parquet 
if not os.path.exists('./Tidy/SNIIM.parquet'):
    df_completo['Observacion'] = df_completo['Observacion'].astype(str)
    df_completo.to_parquet('./Tidy/SNIIM.parquet', index=False)

In [52]:
#df_completo.columns.to_list()